<a href="https://colab.research.google.com/github/quanndm/kltn/blob/dev/notebooks/liver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Init - import library - download data

In [1]:
!rm -rf kltn
!rm -rf sample_data
!git clone -b dev https://github.com/quanndm/kltn.git

Cloning into 'kltn'...
remote: Enumerating objects: 1319, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 1319 (delta 69), reused 49 (delta 26), pack-reused 1212 (from 2)
Receiving objects: 100% (1319/1319), 13.44 MiB | 8.06 MiB/s, done.
Resolving deltas: 100% (857/857), done.


In [2]:
from kltn.init.install_dependencies import install_packages, load_config
install_packages("./kltn")
config = load_config("./kltn/parameters.yaml")

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive

from kltn import *
from kltn.datasets.get_datasets import get_datasets_lits
from kltn.utils.logger import init_logger
from kltn.utils.train_val_epochs import  trainer_stage1
from kltn.utils.metrics import  DiceMetric, TverskyLossWSigmoid, DiceLossWSigmoid
from kltn.models.model_factory import ModelFactory
from kltn.utils.visualize_results import visualize_results

In [ ]:
drive.flush_and_unmount()
drive.mount("/content/drive")

In [ ]:
LOGGER = init_logger(config["log_path"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from kltn.datasets.prepare_data import prepare_dataset_LiTS
prepare_dataset_LiTS(dir_name=config["source_folder_lits"])

## Load data

In [ ]:
full_train_dataset, val_dataset = get_datasets_lits(source_folder=config["source_folder_lits"], seed=123, fold_number=2, mode="liver")
print(len(full_train_dataset), len(val_dataset))

105 26


In [ ]:
train_loader = torch.utils.data.DataLoader(full_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=1, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                         pin_memory=True, num_workers=1)

## Training

In [ ]:
model = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)

In [ ]:
criterion = DiceLossWSigmoid().to(device)

dice_acc = DiceMetric(include_background=False, reduction='mean_batch', get_not_nans=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=float(config["learning_rate"]), weight_decay=float(config["weight_decay"]))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(config["max_epochs"]), eta_min = float(config["eta_min"]))

In [ ]:
LOGGER.info("[TRAINER] Start TRAIN process...")

(
    val_acc_max,
    best_epoch,
    dices_liver,
    loss_epochs,
    trains_epoch,
    ious_liver,
    precisions_liver,
    recalls_liver,
    time_tmp
) = trainer_stage1(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_func=criterion,
    acc_func=dice_acc,
    scheduler=scheduler,
    batch_size=config["batch_size"],
    max_epochs = config["max_epochs"],
    start_epoch = config["start_epoch"],
    val_every=config["val_every"],
    path_save_model=config["path_save_model_state"],
    logger=LOGGER,
    save_model=True,
    post_fix="liver"
)

[TRAINER] Start TRAIN process...
INFO:kltn.utils.logger:[TRAINER] Start TRAIN process...

==============================Training epoch 1==============================
INFO:kltn.utils.logger:
==============================Training epoch 1==============================


Epoch 1/5 1/105 loss: 0.7993 time 3.20s
Epoch 1/5 2/105 loss: 0.7768 time 1.22s
Epoch 1/5 3/105 loss: 0.7174 time 5.65s
Epoch 1/5 4/105 loss: 0.7211 time 4.09s
Epoch 1/5 5/105 loss: 0.7074 time 1.49s
Epoch 1/5 6/105 loss: 0.7162 time 4.69s
Epoch 1/5 7/105 loss: 0.6581 time 5.10s
Epoch 1/5 8/105 loss: 0.6257 time 4.69s
Epoch 1/5 9/105 loss: 0.6319 time 1.07s
Epoch 1/5 10/105 loss: 0.6270 time 2.81s
Epoch 1/5 11/105 loss: 0.6170 time 2.13s
Epoch 1/5 12/105 loss: 0.6070 time 4.93s
Epoch 1/5 13/105 loss: 0.5846 time 4.55s
Epoch 1/5 14/105 loss: 0.5717 time 5.49s
Epoch 1/5 15/105 loss: 0.5527 time 4.04s
Epoch 1/5 16/105 loss: 0.5349 time 8.45s
Epoch 1/5 17/105 loss: 0.5219 time 1.33s
Epoch 1/5 18/105 loss: 0.5051 time 5.05s
Epoch 1/5 19/105 loss: 0.4893 time 2.21s
Epoch 1/5 20/105 loss: 0.4755 time 1.59s
Epoch 1/5 21/105 loss: 0.4629 time 0.94s
Epoch 1/5 22/105 loss: 0.4499 time 6.32s
Epoch 1/5 23/105 loss: 0.4418 time 5.24s
Epoch 1/5 24/105 loss: 0.4451 time 4.23s
Epoch 1/5 25/105 loss: 0.

Final training epochs: 1/5 ---[loss: 0.2352] ---[time 367.07s]
INFO:kltn.utils.logger:Final training epochs: 1/5 ---[loss: 0.2352] ---[time 367.07s]

********************Epoch 1 Validation********************
INFO:kltn.utils.logger:
********************Epoch 1 Validation********************


Epoch 1/5 105/105 loss: 0.2352 time 3.32s


Val 1/5 1/26, Dice_Liver: 0.865247, time 2.97s
INFO:kltn.utils.logger:Val 1/5 1/26, Dice_Liver: 0.865247, time 2.97s
Val 1/5 2/26, Dice_Liver: 0.919390, time 0.16s
INFO:kltn.utils.logger:Val 1/5 2/26, Dice_Liver: 0.919390, time 0.16s
Val 1/5 3/26, Dice_Liver: 0.883482, time 4.77s
INFO:kltn.utils.logger:Val 1/5 3/26, Dice_Liver: 0.883482, time 4.77s
Val 1/5 4/26, Dice_Liver: 0.893390, time 6.48s
INFO:kltn.utils.logger:Val 1/5 4/26, Dice_Liver: 0.893390, time 6.48s
Val 1/5 5/26, Dice_Liver: 0.912058, time 0.63s
INFO:kltn.utils.logger:Val 1/5 5/26, Dice_Liver: 0.912058, time 0.63s
Val 1/5 6/26, Dice_Liver: 0.898304, time 4.35s
INFO:kltn.utils.logger:Val 1/5 6/26, Dice_Liver: 0.898304, time 4.35s
Val 1/5 7/26, Dice_Liver: 0.920769, time 4.33s
INFO:kltn.utils.logger:Val 1/5 7/26, Dice_Liver: 0.920769, time 4.33s
Val 1/5 8/26, Dice_Liver: 0.873534, time 3.29s
INFO:kltn.utils.logger:Val 1/5 8/26, Dice_Liver: 0.873534, time 3.29s
Val 1/5 9/26, Dice_Liver: 0.904651, time 0.56s
INFO:kltn.utils.l

New best (0.000000 --> 0.883954). At epoch 1
Epoch 2/5 1/105 loss: 0.0919 time 4.54s
Epoch 2/5 2/105 loss: 0.1004 time 3.99s
Epoch 2/5 3/105 loss: 0.1161 time 1.06s
Epoch 2/5 4/105 loss: 0.1177 time 2.06s
Epoch 2/5 5/105 loss: 0.1163 time 5.04s
Epoch 2/5 6/105 loss: 0.1111 time 4.67s
Epoch 2/5 7/105 loss: 0.1136 time 5.01s
Epoch 2/5 8/105 loss: 0.1160 time 5.28s
Epoch 2/5 9/105 loss: 0.1177 time 0.68s
Epoch 2/5 10/105 loss: 0.1138 time 0.69s
Epoch 2/5 11/105 loss: 0.1173 time 1.59s
Epoch 2/5 12/105 loss: 0.1160 time 4.44s
Epoch 2/5 13/105 loss: 0.1137 time 4.85s
Epoch 2/5 14/105 loss: 0.1122 time 1.79s
Epoch 2/5 15/105 loss: 0.1132 time 1.23s
Epoch 2/5 16/105 loss: 0.1126 time 1.59s
Epoch 2/5 17/105 loss: 0.1177 time 0.68s
Epoch 2/5 18/105 loss: 0.1220 time 3.01s
Epoch 2/5 19/105 loss: 0.1221 time 1.52s
Epoch 2/5 20/105 loss: 0.1201 time 4.87s
Epoch 2/5 21/105 loss: 0.1225 time 1.98s
Epoch 2/5 22/105 loss: 0.1225 time 1.21s
Epoch 2/5 23/105 loss: 0.1270 time 3.05s
Epoch 2/5 24/105 loss

In [ ]:
import json
path_save_result = f"/content/drive/MyDrive/KLTN/code/result_model_{model.__class__.__name__}_liver.json"

results = {
    "dice_liver": [float(x) for x in np.array(dices_liver, dtype=np.float32)],
    "loss": [float(x) for x in np.array(loss_epochs, dtype= np.float32)],
    "iou_liver": [float(x) for x in np.array(ious_liver, dtype=np.float32)],
    "precision_liver": [float(x) for x in np.array(precisions_liver, dtype=np.float32)],
    "recall_liver": [float(x) for x in np.array(recalls_liver, dtype=np.float32)],
    "best_epoch": best_epoch,
    "time_train": time_tmp
}
with open(path_save_result, "w") as f:
    json.dump(results, f, indent=4)